# Install Transformers Library

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install transformers

     |████████████████████████████████| 2.5 MB 5.5 MB/s 
     |████████████████████████████████| 3.3 MB 42.3 MB/s 
     |████████████████████████████████| 895 kB 51.2 MB/s 


In [9]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
# from transformers import AutoModel, BertTokenizerFast
from transformers import RobertaTokenizer, RobertaModel
# specify GPU
# device = torch.device("cuda")

In [10]:
df = pd.read_json('/content/drive/MyDrive/sarcasm detection/Datasets/reddit/sarcasm_detection_shared_task_reddit_training.jsonl',lines=True)
df1 = pd.read_json('/content/drive/MyDrive/sarcasm detection/Datasets/reddit/sarcasm_detection_shared_task_reddit_testing.jsonl',lines=True)

df['labels'] = df['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1['labels'] = df1['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1.tail()

,label,context,response,id,labels
1795,SARCASM,"[Spoiler Cristiane Justino vs. Amanda Nunes, H...",she will probably beat him too. she can fight ...,reddit_1796,0
1796,NOT_SARCASM,[Treyarch makes the most unbalanced game ever....,"No, but then the game really glitch(ed) out fo...",reddit_1797,1
1797,SARCASM,"[First Official Image from ""Zombieland: Double...",> Zombieland ~~2:~~ Double Tap They changed it...,reddit_1798,0
1798,SARCASM,[Its time to ban /r/The_Donald: Calling out sp...,We're not *supporting racists* and prospective...,reddit_1799,0
1799,SARCASM,[59 Alabama ministers sign a letter saying Roy...,Is she old enough to have Facebook,reddit_1800,0


In [30]:
df.loc[10,'context']

['@USER if you had a fever would the pancakes be bigger ? maybe even giant size ? ( just asking for a friend with a fever )',
 '@USER she will be so happy to hear it , tho will probably wait until the ability to taste food returns .']

In [ ]:
from sklearn.utils import shuffle
df=shuffle(df,random_state=42)

In [ ]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
!pip install demoji
!pip install contractions
import demoji
import contractions

demoji.download_codes()
nltk.download('stopwords') 
# nltk.download('wordnet')

     |████████████████████████████████| 286kB 10.4MB/s 
     |████████████████████████████████| 327kB 28.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85392 sha256=a54438c62a3bfd12bdffb3123310aef1b3947930704dbf4a12250c84a9732aeb
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
... OK (Got response in 0.14 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation(https://www.nltk.org/_modules/nltk/tokenize/regexp.html)
    text = re.sub(r"@",'', text)
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)
    text = re.sub(r"USER",'@USER', text)
    # text = re.sub(r"USER",'', text)
    text = re.sub(r'<URL>','',text)

    text = demoji.replace_with_desc(text)
    text = re.sub(r':','',text)
    text = contractions.fix(text)
    # text = re.sub(r'\.+','',text)
    # tokeniser = RegexpTokenizer(r'\s+', gaps=True)
    # tokeniser = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    # tokeniser = RegexpTokenizer(r'\w+')
    # tokens = tokeniser.tokenize(text)

    # Lowercase and lemmatise 
    # lemmatiser = WordNetLemmatizer()
    # lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    # keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    
    # stemming
    # ps=PorterStemmer()
    # keywords=[ps.stem(x) for x in keywords]
    return text

def preprocess_text2(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation (optional)
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [ ]:
def fun(A,l):
  if len(A)<=l:
    return A
  else:
    # return A[-l::1]
    return A[-1:-l-1:-1]
def fun1(A):
  return ''.join(A)

In [ ]:
l=0

# df['tweets']= df['response'] +'reply-'+ df['context'].apply(fun,args=[l]).apply(fun1)
# df1['tweets']=df1['response'] +'reply-'+ df1['context'].apply(fun,args=[l]).apply(fun1)






df['tweets']= df['context'].apply(fun,args=[l]).apply(fun1) +'reply-'+ df['response']
df1['tweets']= df1['context'].apply(fun,args=[l]).apply(fun1) +'reply-'+ df1['response']


# df['tweets']= df['context'].apply(fun,args=[l]).apply(fun1) + df['response']
# df1['tweets']= df1['context'].apply(fun,args=[l]).apply(fun1) + df1['response']

df['tweets']= df['tweets'].apply(preprocess_text) 
df1['tweets']=df1['tweets'].apply(preprocess_text)
# df['tweets']=df['tweets']+'reply-'+ df['response'].apply(preprocess_text)
# df1['tweets']=df1['tweets']+'reply-'+ df1['response'].apply(preprocess_text)

In [ ]:
# check class distribution
df['labels'].value_counts(normalize = True)

1    0.5
0    0.5
Name: labels, dtype: float64

In [ ]:
df['tweets']

274     reply-Because Kentucky is such a crucial battl...
3430    reply-Well you have to spin the xray emitter a...
4324    reply-I think the people who complain the most...
1027    reply-does not help when you have legendary me...
1589          reply-it is 2x2 tho, nobody cares about 2x2
                              ...                        
3444    reply-Or we can lead and come up with innovati...
466     reply-Nah, it is all a conspiracy by the Clint...
3092    reply-Do you have a source for how much the NS...
3772    reply-I kind of wish he would go back to a Mit...
860     reply-Of course you do not have a job, you stu...
Name: tweets, Length: 4400, dtype: object

# Split train dataset into train, validation and test sets

In [ ]:

X_s = df['tweets'].values
y_s = df['labels'].values

Xt_s = df1['tweets'].values
yt_s = df1['labels'].values

train_text, temp_text, train_labels, temp_labels = X_s,Xt_s,y_s,yt_s

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)
val_text, test_text, val_labels, test_labels=temp_text, temp_text, temp_labels, temp_labels

# X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.2, random_state=42)


In [ ]:
# pip install -U sentence-transformers

# Import BERT Model and BERT Tokenizer

---



---



In [ ]:
# import BERT-base pretrained model
# bert = AutoModel.from_pretrained('bert-base-uncased')

# # Load the BERT tokenizer
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# import BERT-base pretrained model


# from transformers import RobertaTokenizer, TFRobertaModel
# bert = RobertaModel.from_pretrained('roberta-base')

# # Load the BERT tokenizer
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

from transformers import RobertaTokenizer, TFRobertaModel
bert = RobertaModel.from_pretrained('roberta-large')

# Load the BERT tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

In [ ]:
# sample data
text = ["this is a bert model tutorial", "we will fine-tune a bert model"]
# encode text
sent_id = tokenizer.batch_encode_plus(text, padding=True, return_token_type_ids=False)

In [ ]:
# output
print(sent_id)

# Tokenization

In [ ]:
# # get length of all the messages in the train set
# seq_len = [len(i.split()) for i in train_text]
# for i in train_text:
#   print(i)
# pd.Series(seq_len).hist(bins = 30)

In [ ]:
# max_seq_len = 190
max_seq_len = 128

In [ ]:
total=0
for i in range(5000):
  total+=len(df['tweets'][i].split(' '))
total/5000

In [ ]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# Convert Integer Sequences to Tensors

In [ ]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

# Create DataLoaders

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 16

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)#,shuffle=True)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

# Freeze BERT Parameters

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = True
# print(bert)

# Define Model Architecture

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      # self.fc1 = nn.Linear(768,256)
      self.fc1 = nn.Linear(1024,256)
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(256,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id,mask, return_dict=False)
      # print("dfghjjhfchgbhlvmb jfbvfddvfgbhnjhgvcvtynubtrvcebnuexcvbnbtvrcedcvbnjbnjn11111 ",cls_hs.shape)
      x = self.fc1(cls_hs)
      # print("dfghjjhfchgbhlvmb jfbvfddvfgbhnjhgvcvtynubtrvcebnuexcvbnbtvrcedcvbnjbnjn22222 ",x.shape)
      x = self.relu(x)
      # print("dfghjjhfchgbhlvmb jfbvfddvfgbhnjhgvcvtynubtrvcebnuexcvbnbtvrcedcvbnjbnjn22222 ",x.shape)
      x = self.dropout(x)
      # print("dfghjjhfchgbhlvmb jfbvfddvfgbhnjhgvcvtynubtrvcebnuexcvbnbtvrcedcvbnjbnjn22222 ",x.shape)
      # output layer
      x = self.fc2(x)
      # print("dfghjjhfchgbhlvmb jfbvfddvfgbhnjhgvcvtynubtrvcebnuexcvbnbtvrcedcvbnjbnjn22222 ",x.shape)
      # apply softmax activation
      x = self.softmax(x)
      # print("dfghjjhfchgbhlvmb jfbvfddvfgbhnjhgvcvtynubtrvcebnuexcvbnbtvrcedcvbnjbnjn22222 ",x.shape)
      return x

In [ ]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 2e-5)

# Find Class Weights

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print(class_wts)

[1. 1.]


In [ ]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 10

# Fine-Tune BERT

In [ ]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
!nvidia-smi

Mon Jul 12 10:21:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    63W / 149W |   1816MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Start Model Training

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
epochs=1
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss,_ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    # if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
  
    # get predictions for test data
    !nvidia-smi
    # with torch.no_grad():
    0.
    #   preds = model(test_seq.to(device), test_mask.to(device))
    #   preds = preds.detach().cpu().numpy()
    # preds = np.argmax(preds, axis = 1)
    # print(classification_report(test_y, preds))
    # print(accuracy_score(test_y, preds))

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 1
  Batch    50  of    313.
  Batch   100  of    313.
  Batch   150  of    313.
  Batch   200  of    313.
  Batch   250  of    313.
  Batch   300  of    313.

Evaluating...
  Batch    50  of    113.
  Batch   100  of    113.
Mon Jul 12 11:07:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    65W / 149W |  10591MiB / 11441MiB |      0%      Default |
|          

# Load Saved Model

In [ ]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

# Get Predictions for Test Data

In [ ]:
pred=[]
with torch.no_grad():
  for i in range(0,90):
    preds = model(test_seq[i*20:i*20+20].to(device), test_mask[i*20:i*20+20].to(device))
    preds = preds.detach().cpu().numpy()
    pred.append(preds)
print(np.array(pred).shape)

(90, 20, 2)


In [ ]:
pred1=np.array(pred)
# pred1

In [ ]:
pred2=pred1.reshape((1800,2))
pred2.shape

(1800, 2)

In [ ]:
p=pd.DataFrame(pred2)
p.to_csv('context_2.csv')

In [ ]:
pred3 = np.argmax(pred2, axis = 1)
pred3 

array([1, 0, 0, ..., 0, 1, 1])

In [ ]:
print(classification_report(test_y, pred3))

              precision    recall  f1-score   support

           0       0.67      0.86      0.75       900
           1       0.80      0.57      0.67       900

    accuracy                           0.71      1800
   macro avg       0.74      0.72      0.71      1800
weighted avg       0.74      0.71      0.71      1800



In [ ]:
# confusion matrix
# print(pred)
pd.crosstab(test_y, pred3)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

print(accuracy_score(test_y, pred3))

0.715


In [ ]:
import scipy

In [ ]:
brcc=pd.read_csv('76.6_roberta_large_100_32_2e-5_4epoch_preprocessing+@reply+user.csv')
brcc=np.array(brcc[['0','1']])
brcc=scipy.special.softmax(brcc,axis=1)
# bsvm=np.multiply(bsvm,-1)
brcc

FileNotFoundError: ignored

In [ ]:
brc=pd.read_csv('77.44_1context+response_batch16+3epoch_180maxlen.csv')
brc=np.array(brc[['0','1']])
brc=scipy.special.softmax(brc,axis=1)
# bsvm=np.multiply(bsvm,-1)
brc

In [ ]:
br=pd.read_csv('77.7_2context+response_batch16+4epoch_190maxlen.csv')
br=np.array(br[['0','1']])
br=scipy.special.softmax(br,axis=1)
# bsvm=np.multiply(bsvm,-1)
br

In [ ]:
bc=pd.read_csv('76.56_2context_+response_temserflow.csv')
bc=np.array(bc[['0','1']])
bc=scipy.special.softmax(bc,axis=1)
# bsvm=np.multiply(bsvm,-1)
bc

In [ ]:
bkc=pd.read_csv('75_lstm_base_response.csv')
bkc=np.array(bkc[['0','1']])
bkc=scipy.special.softmax(bkc,axis=1)
# bsvm=np.multiply(bsvm,-1)
bkc

In [ ]:
# bertc=np.array(bc[['0','1']])
bertc1=np.multiply(bc,0.6)

# bertrcc=np.array(brcc[['0','1']])
bertrcc1=np.multiply(brcc,0.8)

# bertcr=np.array(brc[['0','1']])
bertcr1=np.multiply(brc,0.6)

# bertr=np.array(br[['0','1']])
bertr1=np.multiply(br,0.45)



# # bertc=np.array(bc[['0','1']])
# bertc1=np.multiply(bc,0.4)

# # bertrcc=np.array(brcc[['0','1']])
# bertrcc1=np.multiply(brcc,0.8)

# # bertcr=np.array(brc[['0','1']])
# bertcr1=np.multiply(brc,0.4)

# bertr=np.array(br[['0','1']])
# bertr1=np.multiply(br,0.8)

In [ ]:
yhats = [bertc1,bertr1,bertcr1,bertrcc1]
# yhats = [bertcr1,bertrcc1,bertr1]
# yhats = [bertcr1,bertrcc1,bertr1]
# import numpy as np
# yhats = np.array(yhats)

# # sum across ensemble members
# summed = np.sum(yhats, axis=0)
# # argmax across classes
# result = np.argmax(summed, axis=1)

# from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# accuracy_score(result,test_y)

In [ ]:
ans=[]
yhat=[bc,br,brcc,brc,bkc]
# yhat=[brcc,br,brc]
for i in yhat:
  ans.append(np.argmax(i, axis=1))
summed = np.sum(ans, axis=0)/4
res=[]
for i in summed:
  if i<=0.5:
    res.append(0)
  else:
    res.append(1)

print(res)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# accuracy_score(res,test_y)


print(classification_report(test_y, res))
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

print(accuracy_score(test_y, res))

#Ensembling

In [5]:
import pandas as pd
import numpy as np
from scipy import special

In [6]:
b3c=pd.read_csv('/content/drive/MyDrive/sarcasm detection/Results/reddit/71.388_response+3contest_5epoch_120len_32_batch_1e-5lr ().csv')
b3c=np.array(b3c[['0','1']])
b3c=special.softmax(b3c,axis=1)

In [7]:
br=pd.read_csv('/content/drive/MyDrive/sarcasm detection/Results/reddit/70.8_Rresponsereddit_1e-5_5_epoch_learnigrate  anually changed.csv')

br=np.array(br[['0','1']])
br=special.softmax(br,axis=1)

In [16]:
test_y=df1['labels'].values

In [11]:
ans=[]
yhat=[br,b3c]
# yhat=[brcc,br,brc]
for i in yhat:
  ans.append(np.argmax(i, axis=1))
summed = np.sum(ans, axis=0)/len(yhat)
res=[]
for i in summed:
  if i<=0.5:
    res.append(0)
  else:
    res.append(1)

print(res)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support,classification_report
# accuracy_score(res,test_y)


print(classification_report(test_y, res))
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

print(accuracy_score(test_y, res))

[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 

#Augumentation Ensembling

In [13]:

b3c=pd.read_csv('/content/drive/MyDrive/sarcasm detection/Results/arg_twitter/76.38_120len_4epoch_32batch_3context.csv')
b3c=np.array(b3c[['0','1']])
b3c=special.softmax(b3c,axis=1)

In [14]:
br=pd.read_csv('/content/drive/MyDrive/sarcasm detection/Results/arg_twitter/77.27_argumented_32_120_response_4epoch.csv')
br=np.array(br[['0','1']])
br=special.softmax(br,axis=1)

In [17]:
ans=[]
yhat=[br,b3c]
# yhat=[brcc,br,brc]
for i in yhat:
  ans.append(np.argmax(i, axis=1))
summed = np.sum(ans, axis=0)/len(yhat)
res=[]
for i in summed:
  if i<=0.5:
    res.append(0)
  else:
    res.append(1)

print(res)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support,classification_report
# accuracy_score(res,test_y)


print(classification_report(test_y, res))
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

print(accuracy_score(test_y, res))

[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 

In [4]:
dfArg_c=pd.read_csv('/content/drive/MyDrive/sarcasm detection/augumented_data/context1.csv')
dfArg_r=pd.read_csv('/content/drive/MyDrive/sarcasm detection/augumented_data/response3.csv')

array([[0,
        "@ user online don't get it.. think you don't expect, is only would, moved right along.. instead either pretend to bear care of them an troll them.."],
       [1,
        '@ user tries to protest again. talk about jake and his mate think they label with wtf does jones?'],
       [2,
        '@ show the lives about madness about money life moves, oh! # learnhowthesystemworks'],
       ...,
       [4997,
        '@ user *\'rec right... " yes! # protection now impossible # privacy makes not "'],
       [4998,
        '@ user diary - delusional voter who takes the word still not chosen at least value instead of researching : finding out that s / he s off being performed to'],
       [4999,
        "@ user i hope you know no news from this have said anything about that luxurious restaurant ('sokoto on to the east ) passengers kidnapped by armed men on nine may 1987? among them is one of your former patients."]],
      dtype=object)

In [16]:
df = pd.read_json('/content/drive/MyDrive/sarcasm detection/Datasets/twitter/sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)


In [24]:
df['response'][4]

'@USER @USER Pretty Sure the Anti-Lincoln Crowd Claimed That " Democracy Was on the Ballot " in 1860 , too . They Thought Lincoln Was " Authoritarian " . #GOP #PartyOfLincoln #Democrats'

In [25]:
dfArg_r.values[4]

array([4,
       '@ and pretty sure that the a - lincoln word of ( " what was on the ballot " here 1860, too. personal support lincoln was " authoritarian ". # gop # partyoflincoln # action'],
      dtype=object)